<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 08 Prediction Simulation

### Riccardo Spolaor (864877)

</center>

In [276]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import python_files.ModelBuilder as modbuild

In [277]:
%store -r

In [278]:
dataframe = pd.read_csv('./datasets/dataframe05.csv', low_memory = False)
pred_dataframe = pd.read_csv('./datasets/dataframe_pred.csv', low_memory = False)

In [279]:
modbuild.rename_columns(dataframe)

In [280]:
tournament_players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:64]

In [281]:
rf = RandomForestClassifier(n_estimators = forest_best_features['n_estimators'], 
                            max_depth = forest_best_features['max_depth'], n_jobs = -1)
rf.fit(pred_dataframe.drop(['Winner','FadigueTournGamesA>FadigueTournGamesB',
                    'FadigueTournSetsA>FadigueTournSetsB'], axis = 1), pred_dataframe['Winner'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=175,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

Prendo le informazioni dai dataset precedenti, fadigue A B non sono inseribili metto tutto a 0, tengo l'ultima media di game e set vinti dato che non posso calcolarli

MAXFILLED E AVGFILLED MESSI A 1 PERCHE GESTITI IO

NON POsso tenere fadiguea e b

In [282]:
def get_value(p, value):
    try:
        wdf = dataframe[dataframe['PlayerA'] == p].tail(1)[value + 'A']
        ldf = dataframe[dataframe['PlayerB'] == p].tail(1)[value+'B']
        df = wdf.append(ldf.rename(columns = {value+'B':value+'A'}))
        df.sort_index(inplace= True)
        return df.tail(1).values[0]
    except: 
        if value == 'Pts':
            return dataframe[[value+'A',value+'B']].min(axis = 1).min(axis = 0)
        else:
            return dataframe[[value+'A',value+'B']].max(axis = 1).max(axis = 0)

    
def get_odds(p, value):
    try:
        df = dataframe[dataframe['PlayerA'] == p].tail(5)[value+'A'].append(
            dataframe[dataframe['PlayerB'] == p].tail(5)[value+'B'].rename(
                columns = {value+'A': value+'B'}))
        return df.mean(axis = 0)
    except:
        return dataframe[[value + 'A',value + 'B']].mean(axis = 1).mean(axis = 0)
    
def get_gamesMean(p):
    try:
        df = dataframe[dataframe['PlayerA'] == p].tail(5)[['1A', '2A', '3A', '4A', '5A']].append(
            dataframe[dataframe['PlayerB'] == p].tail(5)[['1B', '2B', '3B', '4B', '5B']].rename(
                columns = {'1B': '1A', '2B':'2A', '3B':'3A', '4B':'4A', '5B':'5A'}))
        df.sort_index(inplace= True, axis = 1)
        return df.tail(5).sum(axis = 1).mean(axis = 0)
    except:
        return 0

def get_setsMean(p):
    try:
        df = dataframe[dataframe['PlayerA'] == p].tail(5)['setsA'].append(
            dataframe[dataframe['PlayerB'] == p].tail(5)['setsB'].rename(
                columns = {'setsB': 'setsA'}))
        df.sort_index(inplace= True)
        return df.tail(5).mean(axis = 0)
    except:
        return 0

def get_played(p):
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)['PlayedA']
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)['PlayedB']
    df = wdf.append(ldf.rename(columns = {'PlayedB':'PlayedA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_ratio(p):
    played = get_played(p)
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)['WonRatioA']
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)['WonRatioB']
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)


# OPPONENTS WRATIO A E B CI BASTA RECUPERARLO UNA SOLA VOLTA PERCHE' I GIOCATORI SI AFFRONTANO UNA SOLA VOLTA
def get_opponents_won_ratio(pa, pb):
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.PlayerB ==  pb)][['OpponentsWRatioA','OpponentsWRatioB',
                                                                         'OpponentsPlayed']].tail(1)
    ldf = dataframe[(dataframe.PlayerA == pb) & (dataframe.PlayerB ==  pa)][['OpponentsWRatioA','OpponentsWRatioB', 
                                                                          'OpponentsPlayed']].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return(((ldf['OpponentsWRatioB']*ldf['OpponentsPlayed']))/(ldf['OpponentsPlayed'] + 1)).values[0]
    elif ldf.empty:
        return(((wdf['OpponentsWRatioA']*wdf['OpponentsPlayed']) + 1)/(wdf['OpponentsPlayed'] + 1)).values[0] 
    elif ldf.index.values > wdf.index.values:
        return(((ldf['OpponentsWRatioB']*ldf['OpponentsPlayed']))/(ldf['OpponentsPlayed'] + 1)).values[0]
    else:
        return(((wdf['OpponentsWRatioA']*wdf['OpponentsPlayed']) + 1)/(wdf['OpponentsPlayed'] + 1)).values[0]

def get_retired_last(p):
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return int(ldf['Retired'] == 1)
    elif ldf.empty:
        return 0  
    elif ldf.index.values > wdf.index.values:
        return int(ldf['Retired'] == 1)
    else:
        return 0
    
def get_played_court(p):
    wdf = dataframe[(dataframe.PlayerA == p) & (dataframe.Hard)]['PlayedCourtA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == p) & (dataframe.Hard)]['PlayedCourtB'].tail(1)
    df = wdf.append(ldf.rename(columns = {'PlayedCourtB':'PlayedCourtA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_court(p):
    played = get_played_court(p)
    wdf = dataframe[(dataframe.PlayerA == p) & (dataframe.Hard)].tail(1)['WonRatioCourtA']
    ldf = dataframe[(dataframe.PlayerB == p) & (dataframe.Hard)].tail(1)['WonRatioCourtB']
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)
    
def get_hand(p):
    handdf = dataframe[dataframe['PlayerA'] == p].tail(1)['HandA'].append(
        dataframe[dataframe['PlayerB'] == p].tail(1)['HandB'].rename(
            columns = {'HandB': 'HandA'}))
    handdf.sort_index(inplace= True)
    return handdf.tail(1).values[0]

def get_played_vs_same_handed(pa, pb):
    hand= get_hand(pb)
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.HandB ==  hand)]['PlayedVsSameHandedA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == pa) & (dataframe.HandA ==  hand)]['PlayedVsSameHandedB'].tail(1)
    df = wdf.append(ldf.rename(columns = {'PlayedVsSameHandedB':'PlayedVsSameHandedA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_vs_same_handed(pa, pb):
    hand= get_hand(pb)
    played = get_played_vs_same_handed(pa, pb)
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.HandB ==  hand)]['WonRatioVsSameHandedA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == pa) & (dataframe.HandA ==  hand)]['WonRatioVsSameHandedB'].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)

In [283]:
get_won_vs_same_handed('SCHERRER J.C.', 'NADAL R.')

0.0

In [284]:
get_won_court('NADAL R.')

0.7733557229790202

In [285]:
invariate_features = {
    'newcsvID': dataframe['csvID'].max() + 1,
    'rank': {p:get_value(p, 'Rank') for p in tournament_players},
    'max': {p:get_odds(p, 'Max') for p in tournament_players},
    'avg': {p:get_odds(p, 'Avg') for p in tournament_players},
    'pts': {p:get_value(p, 'Pts') for p in tournament_players},
    '5_games_mean': {p:get_gamesMean(p) for p in tournament_players},
    '5_sets_mean': {p:get_setsMean(p) for p in tournament_players},
    'retired': {p:get_retired_last(p) for p in tournament_players},
    'hand': {p:get_hand(p) for p in tournament_players}
}

In [286]:
def get_basic_row(pa,pb,n_round):
    df = pd.DataFrame(data = {
                    'csvID': [invariate_features['newcsvID']], 
                    'ATP': [6], 
                    'Location': [ordered_locations['Melbourne']], 
                    'Tournament': [tournamentsDict['Australian Open']], 
                    'Series': [4], 
                    'Court': [1], 
                    'Round': [n_round],
                    'Best of': [5], 
                    'PtsFilledA': [1], 
                    'PtsFilledB': [1], 
                    'MaxFilledA': [1], 
                    'MaxFilledB': [1],
                    'AvgFilledA': [1], 
                    'AvgFilledB': [1], 
                    'Carpet': [0], 
                    'Clay': [0], 
                    'Grass': [0], 
                    'Hard': [1],
                    'FEDERER R.B': [int(pb=='FEDERER R.')], 
                    'FERRER D.A': [int(pa=='FERRER D.')], 
                    'FERRER D.B': [int(pb=='FERRER D.')],
                    'ROBREDO T.A': [int(pa=='ROBREDO T.')],
                    'ROBREDO T.B': [int(pb=='ROBREDO T.')],
                    'NALBANDIAN D.B': [int(pb=='NALBANDIAN D.')], 
                    'DAVYDENKO N.A': [int(pa=='DAVYDENKO N.')],
                    'DAVYDENKO N.B': [int(pb=='DAVYDENKO N.')],
                    'OtherA': [int(pa not in ['FERRER D.',
                                           'ROBREDO T.',
                                           'DAVYDENKO N.'])], 
                    'OtherB': [int(pb not in ['FEDERER R.',
                                            'FERRER D.'
                                            'ROBREDO T.',
                                            'NALBANDIAN D.',
                                            'DAVYDENKO N.'])],
                    'RankA': [invariate_features['rank'][pa]],
                    'RankB': [invariate_features['rank'][pb]],
                    'MaxA': [invariate_features['max'][pa]],
                    'MaxB': [invariate_features['max'][pb]],
                    'AvgA': [invariate_features['avg'][pa]],
                    'AvgB': [invariate_features['avg'][pb]],
                    'PtsA':  [invariate_features['pts'][pa]],
                    'PtsB':  [invariate_features['pts'][pb]],
                    '5_gamesMeanA': [invariate_features['5_games_mean'][pa]],
                    '5_gamesMeanB': [invariate_features['5_games_mean'][pb]],
                    '5_setsMeanA': [invariate_features['5_sets_mean'][pa]],
                    '5_setsMeanB': [invariate_features['5_sets_mean'][pb]],
                    'PlayerA': [pa],
                    'PlayerB': [pb],
                    'HandA': [invariate_features['hand'][pa]],
                    'HandB': [invariate_features['hand'][pb]],
                    'PlayedA': [get_played(pa)],
                    'PlayedB': [get_played(pb)],
                    'WonRatioA': [get_won_ratio(pa)],
                    'WonRatioB': [get_won_ratio(pb)],
                    'OpponentsWRatioA': [get_opponents_won_ratio(pa, pb)],
                    'OpponentsWRatioB': [get_opponents_won_ratio(pb, pa)],
                    'RetiredLastA': [invariate_features['retired'][pa]],
                    'RetiredLastB': [invariate_features['retired'][pb]],
                    'PlayedCourtA': [get_played_court(pa)],
                    'PlayedCourtB': [get_played_court(pb)],
                    'WonRatioCourtA': [get_won_court(pa)],
                    'WonRatioCourtB': [get_won_court(pb)],
                    'PlayedVsSameHandedA': [get_played_vs_same_handed(pa, pb)],
                    'PlayedVsSameHandedB': [get_played_vs_same_handed(pb, pa)],
                    'WonRatioVsSameHandedA': [get_won_vs_same_handed(pa, pb)],
                    'WonRatioVsSameHandedB': [get_won_vs_same_handed(pb, pa)],
               })
    df['RankA>RankB'] = df.apply(lambda x: int(x['RankA'] > x['RankB']), axis=1)
    df['AvgA>AvgB'] = df.apply(lambda x: int(x['AvgA'] > x['AvgB']), axis=1)
    df['MaxA>MaxB'] = df.apply(lambda x: int(x['MaxA'] > x['MaxB']), axis=1)
    df['PtsA>PtsB'] = df.apply(lambda x : int(x['PtsA'] > x['PtsB']), axis = 1)

    df['OpponentsWRatioA>OpponentsWRatioB'] = df.apply(
        lambda x: int(x['OpponentsWRatioA'] > x['OpponentsWRatioB']), axis=1)

    df['PlayedA>PlayedB'] = df.apply(lambda x: int(x['PlayedA'] > x['PlayedB']), axis=1)
    df['WonRatioA>WonRatioB'] = df.apply(lambda x: int(x['WonRatioA'] > x['WonRatioB']), axis=1)

    df['PlayedCourtA>PlayedCourtB'] = df.apply(lambda x: int(x['PlayedCourtA'] > x['PlayedCourtB']),
                                                             axis=1)
    df['WonRatioCourtA>WonRatioCourtB'] = df.apply(
        lambda x: int(x['WonRatioCourtA'] > x['WonRatioCourtB']), axis=1)

    df['PlayedVsSameHandedA>PlayedVsSameHandedB'] = df.apply(
        lambda x: int(x['PlayedVsSameHandedA'] > x['PlayedVsSameHandedB']), axis=1)
    df['WonRatioVsSameHandedA>WonRatioVsSameHandedB'] = df.apply(
        lambda x: int(x['WonRatioVsSameHandedA'] > x['WonRatioVsSameHandedB']), axis=1)

    df['5_gamesMeanA>5_gamesMeanB'] = df.apply(
        lambda x: int(x['5_gamesMeanA'] > x['5_gamesMeanB']), axis=1)
    df['5_setsMeanA>5_setsMeanB'] = df.apply(
        lambda x: int(x['5_setsMeanA'] > x['5_setsMeanB']), axis=1)
    return df

In [287]:
def predict_result(row):
    winner = rf.predict(row.drop(['WonRatioCourtA', 'WonRatioCourtB', 'WonRatioA', 'WonRatioB', 
                      'OpponentsWRatioA','OpponentsWRatioB', 'RankB', 'RankA',  'PtsA',
                      'PtsB', 'MaxA', 'MaxB', 'AvgA', 'AvgB',  'PlayedA', 'PlayedB','PlayedCourtA', 
                      'PlayedCourtB', 'PlayedVsSameHandedA', 'PlayedVsSameHandedB','WonRatioVsSameHandedA',
                      'WonRatioVsSameHandedB', '5_gamesMeanA', '5_gamesMeanB', '5_setsMeanA', '5_setsMeanB', 'PlayerA',
                      'PlayerB', 'HandA', 'HandB'], axis=1))
    return winner[0]

In [288]:
def get_round_prediction(player_list, n_round):
    
    new_list = []
    round_dict = {0: '1st Round', 1: '2nd Round', 2: '3rd Round', 3: '4th Round', 5: 'Quarterfinals', 6:'Semifinals', 
                  7:'The Final'}
    global dataframe 
    
    for i,d in enumerate(zip(player_list[0::2], player_list[1::2])):
        pa,pb = d
        row = get_basic_row(pa, pb, n_round)
        winner = predict_result(row)
        
        if winner:
            print(round_dict[n_round],'- Partita:', i, '- Vincitore:', pb, '- Perdente:', pa)
    
            col_to_invert = ['Player', 'Rank', 'Pts', 'PtsFilled', 'Max', 'Avg', 'MaxFilled', 'AvgFilled',  
                                   'Played', 'WonRatio','WonRatioCourt', 'PlayedCourt', 'OpponentsWRatio',  
                                   'RetiredLast','5_gamesMean', '5_setsMean',  'PlayedVsSameHanded',
                                   'WonRatioVsSameHanded', 'Hand']
    
            for f in col_to_invert:
                row.rename(columns = {f+'A': f+'B', f+'B': f + 'A'}, inplace= True)
            
            new_list += [pb]
            dataframe = dataframe.append(row, ignore_index = True, sort = False)
        else:
            print(round_dict[n_round],'- Partita:', i, '- Vincitore:', pa, '- Perdente:', pb)
            new_list += [pa]
            dataframe = dataframe.append(row, ignore_index = True, sort = False)
        
    print('\n')
        
    return new_list

In [289]:
from random import shuffle
shuffle(tournament_players)
print (tournament_players)
tournament_players = get_round_prediction(tournament_players, 0)
tournament_players = get_round_prediction(tournament_players, 1)
tournament_players = get_round_prediction(tournament_players, 2)
tournament_players = get_round_prediction(tournament_players, 5)
tournament_players = get_round_prediction(tournament_players, 6)
tournament_players = get_round_prediction(tournament_players, 7)
print('Vincitore:', tournament_players[0])

['FERRER D.', 'MEDVEDEV D.', 'ALMAGRO N.', 'LJUBICIC I.', 'CILIC M.', 'NADAL R.', 'TSITSIPAS S.', 'BLAKE J.', 'NALBANDIAN D.', 'KYRGIOS N.', 'NISHIKORI K.', 'NOVAK J.', 'SRICHAPHAN P.', 'MOYA C.', 'FISH M.', 'FEDERER R.', 'MURRAY A.', 'GAUDIO G.', 'GOFFIN D.', 'DIMITROV G.', 'CORIA G.', 'SIMON G.', 'ZVEREV A.', 'CANAS G.', 'GROSJEAN S.', 'YOUZHNY M.', 'HAAS T.', 'HENMAN T.', 'CHELA J.I.', 'KAFELNIKOV Y.', 'ANDERSON K.', 'RAONIC M.', 'BERDYCH T.', 'KOHLSCHREIBER P.', 'DJOKOVIC N.', 'MONACO J.', 'VERDASCO F.', 'ISNER J.', 'DEL POTRO J. M.', 'FOGNINI F.', 'MONFILS G.', 'ANCIC M.', 'GONZALEZ F.', 'KUERTEN G.', 'QUERREY S.', 'RUSEDSKI G.', 'JOHANSSON T.', 'GASQUET R.', 'WAWRINKA S.', 'RODDICK A.', 'BAUTISTA AGUT R.', 'SAFIN M.', 'STEPANEK R.', 'ROBREDO T.', 'AGASSI A.', 'TSONGA J.W.', 'DAVYDENKO N.', 'THIEM D.', 'HEWITT L.', 'BAGHDATIS M.', 'LOPEZ F.', 'FERRERO J.C.', 'SODERLING R.', 'BAUTISTA R.']
1st Round - Partita: 0 - Vincitore: MEDVEDEV D. - Perdente: FERRER D.
1st Round - Partita: 1 